In [ ]:
# ==================== Colab Setup ====================
# 取消下面的注释以在 Colab 中运行

# 检测是否在 Colab 环境
try:
    import google.colab
    IN_COLAB = True
    print("✓ 运行在 Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ 运行在本地环境")

if IN_COLAB:
    # 方法 1: 从 GitHub 克隆（如果项目已推送）
    # !git clone https://github.com/liuchzzyy/echemistpy.git
    # %cd echemistpy
    
    # 方法 2: 上传压缩包（取消注释以使用）
    # from google.colab import files
    # uploaded = files.upload()  # 上传 echemistpy.zip
    # !unzip -q echemistpy.zip
    # %cd echemistpy
    
    # 安装依赖（使用 pip，因为 Colab 不支持 uv）
    print("\n📦 安装依赖...")
    !pip install -q xarray numpy scipy pandas traitlets matplotlib scikit-learn h5py
    
    # 安装项目本身（开发模式）
    !pip install -q -e .
    
    print("✓ 安装完成！")

# 在 Google Colab 中运行此 Notebook

**方法 1: 从 GitHub 克隆项目（推荐）**

如果项目已推送到 GitHub，取消下面单元格的注释并运行。

**方法 2: 上传项目文件**

如果项目未在 GitHub 上，需要：
1. 将整个项目文件夹压缩为 `echemistpy.zip`
2. 上传到 Colab 并解压
3. 运行下面的安装单元格

# Test STXMAnalyzer with Real Data

This notebook tests the `STXMAnalyzer` in `src/echemistpy/processing/analyzers/stxm.py` using a real TXM HDF5 file.

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# 检测是否在 Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# 设置项目路径
if IN_COLAB:
    # Colab 中，假设已经 cd 到项目根目录
    project_root = Path.cwd()
else:
    # 本地环境，notebook 在 notebooks/ 子目录
    project_root = Path("..").resolve()
    if str(project_root / "src") not in sys.path:
        sys.path.append(str(project_root / "src"))

# Reload modules to pick up changes
import importlib
if 'echemistpy.processing.analyzers.stxm' in sys.modules:
    importlib.reload(sys.modules['echemistpy.processing.analyzers.stxm'])

from echemistpy.io.loaders import load
from echemistpy.processing.analyzers.stxm import STXMAnalyzer

print(f"✓ 项目根目录: {project_root}")

ModuleNotFoundError: No module named 'echemistpy'

In [ ]:
# 1. Load Data
# Using the alpha MnO2 example file
file_path = project_root / "docs" / "examples" / "TXM" / "αMnO2" / "20230629_E1A_749.7x177.5y_specnorm_aliOF.hdf5.hdf5"
print(f"Loading: {file_path}")

raw_data, raw_info = load(file_path)

print("\nRaw Data Info:")
print(raw_info.to_dict())
print("\nRaw Data:")
print(raw_data.data)

In [ ]:
# 2. Analyze
analyzer = STXMAnalyzer(
    pca_components=6,
    perform_clustering=True,
    n_clusters=4,
    # clustering_method="kmeans",
    perform_fitting=True,
)

# Use analyze() method which handles validation, preprocessing, computation, and metadata inheritance
result_data, result_info = analyzer.analyze(raw_data, raw_info)

# 3. Inspect Results
print("\nAnalysis Info Parameters:")
print(result_info.parameters)

print("\nResult Dataset:")
ds_res = result_data.data
print(ds_res)


In [ ]:
# 4. Plot
ds_res = result_data.data

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# PCA Denoised Map (Sum)
if "denoised" in ds_res:
    if "energy_eV" in ds_res.denoised.dims:
        img_sum = ds_res.denoised.sum(dim="energy_eV")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Denoised Sum")
    else:
        ax[0].text(0.5, 0.5, "Denoised has no energy_eV dim")
elif "optical_density" in ds_res:
    if "energy_eV" in ds_res.optical_density.dims:
        img_sum = ds_res.optical_density.sum(dim="energy_eV")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Original Sum")
else:
    ax[0].text(0.5, 0.5, "No Image Data")

# Cluster Labels
if "cluster_labels" in ds_res:
    ax[1].imshow(ds_res.cluster_labels, cmap="tab10")
    ax[1].set_title("Cluster Labels")
else:
    ax[1].text(0.5, 0.5, "No Cluster Labels")

# Cluster Centroids / Spectra
if hasattr(result_info, "others") and "cluster_centroids" in result_info.others:
    centroids = result_info.others["cluster_centroids"]
    if "energy_eV" in ds_res.coords:
        energy = ds_res.energy_eV
        for i, cent in enumerate(centroids):
            ax[2].plot(energy, cent, label=f"Cluster {i}")

        ax[2].legend()
        ax[2].set_title("Cluster Spectra")
    else:
        ax[2].text(0.5, 0.5, "No energy_eV axis")
else:
    ax[2].text(0.5, 0.5, "No centroids")

plt.tight_layout()
plt.show()
